In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

### Выбор метрики

Так как наша модель обчается принимать решения о болезнях людей, то важнее минимизировать ошибку  false positive - решение, что человек не болен, когда он болен на самом деле. Следовательно.....

## Preprosessing

In [115]:
raw_path = '../data/raw/'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [114]:
######
train = train.set_index('ID')
test = test.set_index('ID')

In [116]:
train = train.fillna(0)
train.isnull().values.any()


False

In [117]:
train.head()

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,...,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,...,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0


In [111]:
all_cols = ['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия',
       'Образование', 'Профессия', 'Вы работаете?', 'Выход на пенсию',
       'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
       'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Статус Курения', 'Возраст курения', 'Сигарет в день',
       'Пассивное курение', 'Частота пасс кур', 'Алкоголь', 'Возраст алког',
       'Время засыпания', 'Время пробуждения', 'Сон после обеда',
       'Спорт, клубы', 'Религия, клубы']
category_cols = ['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование']
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']


In [118]:
from sklearn.model_selection import train_test_split

train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]


In [120]:
def join_target_cols(data, TARGET_COLS):
    new_vector = [0] * len(data[TARGET_COLS[0]])
    for i in range(len(TARGET_COLS)):
        column = TARGET_COLS[i]
        for j in range(len(data[column])):
            if data[column][j]:
                if new_vector[j]:
                    new_vector[j] += i**3
                else:
                    new_vector[j] += i+1
    return pd.DataFrame(new_vector)
vec_target = join_target_cols(target, TARGET_COLS)

In [126]:
from category_encoders import *
enc = TargetEncoder(category_cols).fit(train, vec_target)

c:\Users\Кристина\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
c:\Users\Кристина\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [128]:
numeric_dataset = enc.transform(train)

In [129]:
X1_train, X1_val, y_train, y_val = train_test_split(numeric_dataset, vec_target)

In [22]:
##from catboost import CatBoost, CatBoostRegressor, Pool
#train_pool = Pool(X_train, label=y_train,
#                  cat_features=category_cols, feature_names=all_cols)